In [ ]:
import pandas as pd
import numpy as np
import tabula
import glob
import tkinter as tk
import tkinter.messagebox
import PyPDF2


#Crea una lista de todos los archivos .PDF que haya en la carpeta
pdf_files = glob.glob("*.pdf")


#Crea una variable a partir del archivo Excel "Comprobantes"
archivo_excel = pd.ExcelFile("Comprobantes.xlsx")


#Crea una lista de todas las hojas que haya en el archivo Excel "Comprobantes"
excel_hoja_nombre = archivo_excel.sheet_names


#Lista vacia para identificar las pestañas que poseen un nombre distinto a "TEMPLATE"
consolidado_excel_hoja_nombre = []


#Iniciamos el ciclo for para consultar pestaña por pestaña
for template in excel_hoja_nombre:
     
    #Nos aseguramos que se trabajen solo aquellas pestañas que tengan un nombre distinto a "TEMPLATE"
    if not template.startswith("TEMPLATE"):
        
        consolidado_excel_hoja_nombre.append(template)



#Crea una lista completa que contenga la suma de todos los archivos .PDF y hojas en el Excel "Comprobantes"
archivos = pdf_files + consolidado_excel_hoja_nombre         


#Antes de iniciar el ciclo "for" creamos una lista vacia para almacenar los DataFrames que se generen en cada ciclo
#Posteriormente de esta forma obtendremos un total general por jurisdicciones
dfs_consolidado_retenciones = []

dfs_consolidado_comprobantes = []

#Lista vacia para totales calculados (total + impuestos) 
dfs_consolidado_totales = []

#Lista vacia para totales leidos de cada .PDF
dfs_consolidado_totales_pdf = []

#Lista vacia para identificar que comprobantes poseen detalle de iibb
dfs_consolidado_iibb = []


#Comienza un ciclo for para iterar sobre cada uno de los archivos listados en el paso anterior
for archivo in archivos:
    
    
    #Valida si el archivo finaliza con .PDF
    if archivo.endswith('.PDF'):
        
            
        ###############################
        ########## TABULA PY ##########
        ###############################


        #Definimos las areas, dentro del cuerpo del comprobante, de las cuales se va a obtener la informacion:

        # y1 = top (desde el margen superior de la hoja hasta donde comienza la tabla)

        # x1 = left (desde el margen izquierdo de la hoja hasta donde comienza la tabla)

        # y2 = top + height (desde el margen superior de la hoja hasta donde finaliza la tabla)

        # x2 = left + width (desde el margen izquierdo de la hoja hasta donde finaliza la tabla)


   
        #Iniciamos un ciclo for para recorrer todas las hojas del .PDF
        #En cada hoja solo tomaremos el cuerpo de la factura (jurisdicciones e impuestos) y excluiremos el N° de comprobante y TOTAl
        #Luego crearemos una lista con todos los DF'S obtenidos y los concatenaremos para crear uno solo
        
        
        #Lamentablemente "tabula-py" no tiene una funcion directa para contar el numero de paginas en un .PDF
        #Sin embargo combinamos "PyPDF" con "tabula-py" para lograr esto
        file = open(archivo, 'rb')
        pdf_reader = PyPDF2.PdfReader(file)
        #Obtenemos la cantidad total de paginas del .PDF
        numero_paginas = len(pdf_reader.pages)

        
        #Creamos una lista vacia, para listar los DF´s de cada pagina cuando utilicemos .append
        df = []
        
        
        for pagina in range(1, numero_paginas+1):

            # ..luego el orden es el siguiente: y1,x1,y2,x2
            # Debido a que las medidas pueden variar entre un comprobante y otro, se establece el area en porcentaje
            dfs = tabula.read_pdf(archivo, pages=pagina, encoding = 'latin1', relative_area = True, area = [24,2,62,100], stream=True)

            df_pagina = dfs[0]
            
            df.append(df_pagina)
            
            
        df = pd.concat(df).reset_index()
        

        #Accedemos al N° de comprobante
        dfs_1 = tabula.read_pdf(archivo, pages=1, encoding = 'latin1', relative_area = True, area = [1,60,6,100], stream=True)

        dfs_1 = dfs_1[0]
        
        
        #Creamos el nombre de la hoja de Excel
        nombre_hoja = dfs_1.iloc[0,0]


        #Accedemos al importe "TOTAL"
        dfs_2 = tabula.read_pdf(archivo, pages=numero_paginas, encoding = 'latin1', relative_area = True, area = [73,81,77,100], stream=True)

        #El dato obtenido PANDAS lo interpreta como el titulo de la columna
        dfs_2 = dfs_2[0]

        #LLamamos a los nombres de las columnas y los convertimos a una lista para poder trabajarlos
        a = dfs_2.columns.tolist()

        #Creamos un diccionario y le asignamos titulo a la lista
        dfs_2 = {"TOTAL": a}

        #Convertimos el diccionario a un DataFrame
        df_total = pd.DataFrame(dfs_2)
        
         
        #Completamos con el string "0.00" aquellos valores NaN
        #Seleccionamos las columnas a completar
        columnas_a_completar = ['Imp. Gravado', 'Imp. Exento', 'Total']
        df[columnas_a_completar] = df[columnas_a_completar].fillna('0.00')
        
        
        #Completamos con 0 la columna 'ITEM'
        #Primero completamos los NaN con notacion float y luego los convertimos a int
        df['ITEM'] = df['ITEM'].fillna('0.00')
        df['ITEM'] = df['ITEM'].astype(float).astype('int64')
        

        #Convertimos a float la columna 'Imp. Gravado'
        df['Imp. Gravado'] = df['Imp. Gravado'].str.replace('.','')
        df['Imp. Gravado'] = df['Imp. Gravado'].str.replace(',','.').astype(float)


        #Convertimos a float la columna 'Imp. Exento'
        df['Imp. Exento'] = df['Imp. Exento'].str.replace('.','')
        df['Imp. Exento'] = df['Imp. Exento'].str.replace(',','.').astype(float)


        #Convertimos a float la columna 'Total'
        df['Total'] = df['Total'].str.replace('.','')
        df['Total'] = df['Total'].str.replace(',','.').astype(float)


        #Convertimos a float el 'TOTAL' factura leido en el .PDF
        df_total['TOTAL'] = df_total['TOTAL'].str.replace('.','')
        df_total['TOTAL'] = df_total['TOTAL'].str.replace(',','.').astype(float)
        
        
    else:
        
        ######################################
        ############ Comprobantes ############
        ######################################
        
        
        #Convierte a DF el cuadro de la hoja
        df = pd.read_excel('Comprobantes.xlsx', sheet_name = archivo, thousands=',')
        
        #Obtiene el valor total de la ultima fila y la columna "Total" 
        ttal = df.iloc[-1,4]
        
        
        #Se eliminan todas las filas que contegan NaN en la columna "DETALLE" 
        df = df.dropna(subset=['DETALLE'])
        
        #Obtiene el numero de comprobante a partir del nombre de la hoja
        nombre_hoja = archivo
            
        #Creamos un diccionario y le asignamos titulo a la lista
        dfs_2 = {"TOTAL": [ttal]}

        #Convertimos el diccionario a un DataFrame
        df_total = pd.DataFrame(dfs_2)
        

        
    #Accedemos a la tabla que contiene la informacion que relaciona las facturas con sus respectivas NC/ND
    #Las NC se ingresan en negativo, mientras que las ND en positivo
    asociados = pd.read_excel('Ref_comprobante.xlsx')    
    
    
    #Accedemos a la informacion impositiva de la tabla "LFBW" (SAP)
    df_lfbw = pd.read_excel('LFBW.xlsx', usecols = ['Tipo retenciones', 'Ret'])
    

    #Creamos una lista con las jurisdicciones
    lista_prov = ['BUENOS AIRES', 'CAPITAL FEDERAL', 'CATAMARCA', 'CHACO', 'CHUBUT', 'CORDOBA', 'CORRIENTES', 
                 'ENTRE RIOS', 'FORMOSA', 'JUJUY', 'LA PAMPA', 'LA RIOJA', 'MENDOZA', 'MISIONES', 'NEUQUEN',
                 'RIO NEGRO', 'SALTA', 'SAN JUAN', 'SAN LUIS', 'SANTA CRUZ', 'SANTA FE', 'SANTIAGO DEL ESTERO',
                  'TIERRA DEL FUEGO', 'TUCUMAN']


    #', '.join concatena todos los strings por una coma y un espacio
    #Seleccionamos la jurisdiccion dentro del texto en "DETALLE"
    #Consulta si la jurisdiccion de la columna "DETALLE" se encuentra en la lista "lista_prov"
    #.upper() convierte el nombre de las provincias a mayuscula, para que asi lo pueda comparar con la lista "lista_prov"
    df['PROVINCIA'] = df['DETALLE'].apply(lambda provincia: ', '.join([word for word in lista_prov if word in provincia.upper()]))


    df = df[['ITEM', 'DETALLE', 'Imp. Gravado', 'Imp. Exento', 'Total', 'PROVINCIA']]


    #Encontramos el indice o la posicion en la cual se encuentra la palabra "SUBTOTAL GENERAL"
    subtotal_general = df['DETALLE'].tolist().index('SUBTOTAL GENERAL')


    #Creo un DataFrame que contenga solo los datos de la fila 'SUBTOTAL GENERAL'
    df_subtotal_general = df.loc[df['DETALLE'].isin(['SUBTOTAL GENERAL'])]


    #Obtenemos una lista con los ITEM's de las perepciones
    impuestos = df['DETALLE'].tolist()[(subtotal_general+1):]


    #Creo un DataFrame a partir de las jurisdicciones
    df_impuestos = df.loc[df['DETALLE'].isin(impuestos)]


    #Generamos una copia del DataFrame para que no informe error 
    df_impuestos = df_impuestos.copy()


    #Obtenemos una lista con los ITEM's de las jurisdicciones
#    jurisdicciones = df['ITEM'].tolist()[0:subtotal_general]
    jurisdicciones = df['DETALLE'].tolist()[0:subtotal_general]

    #Creo un DataFrame a partir de las jurisdicciones
#    df_jurisdicciones = df.loc[df['ITEM'].isin(jurisdicciones)]
    df_jurisdicciones = df.loc[df['DETALLE'].isin(jurisdicciones)] 

    
    #Cuenta la cantidad de filas del DataFrame
    cantidad_lineas = len(df_jurisdicciones['DETALLE'])


    #Generamos una copia del DataFrame para que no informe error
    df_jurisdicciones = df_jurisdicciones.copy()
    

    #Particionamos el DF para asi tener una mejor representacion grafica en Excel
    #Seleccionamos todas las columnas menos la ultima
    df_jurisdicciones_2 = df_jurisdicciones.iloc[:,0:5].copy()
    #Seleccionamos la ultima columna
    df_jurisdicciones_3 = df_jurisdicciones.iloc[:,5].copy()


    #la función fillna() completa solo los valores nulos de la columna 'Total'
    df_impuestos['Total'] = df_impuestos['Total'].fillna(0)


    #Seleccionamos las columnas del DataFrame que vamos a utilizar
    df_impuestos = df_impuestos[['ITEM', 'DETALLE', 'Total']]


    #Creamos un diccionario para definir el tipo de impuesto que le corresponde a cada "ITEM"
    #Posteriormente se podran agregar mas valores
    clase_impuesto = {
        "ITEM": [10002, 10008, 10096, 10098, 10099, 10127, 10228, 32001],
        "IMPUESTO": ["PERCEPCION", "IVA 10,5%", "PERCEPCION", "PERCEPCION", "PERCEPCION", "PERCEPCION", "PERCEPCION",
                    "PERCEPCION"]
    }


    #A partir del diccionario creamos un DataFrame
    df_clase_impuesto = pd.DataFrame(clase_impuesto)


    #Hacemos una union de DF para obtener el impuesto que le corresponde a cada "ITEM"
    df_impuestos = pd.merge(df_impuestos, df_clase_impuesto, on='ITEM', how='left')


    #Particionamos el DF para asi tener una mejor representacion grafica en Excel
    df_impuestos_2 = df_impuestos[['ITEM', 'DETALLE']].copy()
    df_impuestos_3 = df_impuestos[['Total']].copy()
    df_clase_impuesto_2 = df_impuestos[['IMPUESTO']]


    #Al aplicar groupby totalizamos cada uno de los impuestos segun su tipo
    df_impuestos_total = df_impuestos.groupby('IMPUESTO')[['Total']].sum().reset_index()

    # a = df_percepciones['Total'].sum().round(2)


    df_jurisdicciones_group = df_jurisdicciones.groupby('PROVINCIA')[['Imp. Gravado', 'Imp. Exento', 'Total']].sum().reset_index()


    #Creamos un diccionario para definir el "TIPO RETENCION" y el "TIPO INDICADOR" para las jurisdicciones que NO tienen padron
    #Posteriormente se podran modificar los valores
    retencion = {
        "Tipo retenciones": ["I1", "ID", "IG", "IH", "II", "IK", "IL", "IN", "IO", "IW", "SA", "TG"],
        "Ret": ["18", "03", "03", "12", "05", "07", "04", "02", "01", "01", "02", "01"]
    }


    #A partir del diccionario creamos un DataFrame
    df_retencion = pd.DataFrame(retencion)


    #Filtramos la tabla LFBW para excluir los NaN y asi obtener los indicadores que se actualizan por padron
    #NaN no es igual a nada, por eso no se puede filtrar como texto. Se utiliza para ello isnull
    df_lfbw_padron = df_lfbw.loc[~pd.isnull(df_lfbw.Ret)]


    #Unificamos los DF "df_retencion" y "df_lfbw_padron" a fin de contar con los indicadores para todos los tipos de retencion
    #Luego lo ordenamos por "Tipo retenciones"
    df_retencion_unificado = pd.concat([df_retencion, df_lfbw_padron], axis =0).sort_values('Tipo retenciones')
    
    
        #Creamos un diccionario para definir el "TIPO RETENCION" y el "TIPO INDICADOR" para las jurisdicciones que NO tienen padron
    #Posteriormente se podran modificar los valores
    tipo_retenciones = {
        "Tipo retenciones": ["0X",
                             "GA",
                             "I1",
                             "IA",
                             "IB",
                             "IC",
                             "ID",
                             "IF",
                             "IG",
                             "IH",
                             "II",
                             "IK",
                             "IL",
                             "IM",
                             "IN",
                             "IO",
                             "IT",
                             "IV",
                             "IW",
                             "IX",
                             "IY",
                             "SA",
                             "SD",
                             "TF",
                             "TG"],
        "PROVINCIA": ["MUNICIPALIDAD CORDOBA",
                "GANANCIAS RG. 830",      
                "SANTA FE",
                "SALTA",
                "BUENOS AIRES",
                "CAPITAL FEDERAL",
                "SAN LUIS",
                "FORMOSA",
                "SANTIAGO DEL ESTERO",
                "CHACO",
                "RIO NEGRO",
                "CATAMARCA",
                "LA PAMPA",
                "MENDOZA",
                "MISIONES",
                "LA RIOJA",
                "TUCUMAN",
                "IVA RG. 18",
                "CORRIENTES",
                "CORDOBA",
                "JUJUY",
                "SANTA CRUZ",
                "ENTRE RIOS",
                "TIERRA DEL FUEGO",
                "ADIC. TIERRA DEL FUEGO"
               ]}

   
    #A partir del diccionario creamos un DataFrame
    df_tipo_retenciones = pd.DataFrame(tipo_retenciones)


    #Hacemos una union de DF para obtener la provincia que le corresponde a cada 'Tipo retenciones'
    df_tipo_retenciones = pd.merge(df_retencion_unificado, df_tipo_retenciones, on='Tipo retenciones', how='left')
    
    
    #Hacemos una union de DF para obtener la tabla definitiva a ingresar en SAP, con los 'Tipo retenciones' que 
    #le corresponden a cada provincia
    df_base_retencion = pd.merge(df_jurisdicciones_group[['PROVINCIA']], df_tipo_retenciones, on='PROVINCIA', how='left')
    
    
    #El dato maestro trae el tipo de riesgo segun padron
    # Prueba logica que determina el indicador en base al tipo de riesgo en la provincia de SALTA
    df_base_retencion.loc[((df_base_retencion['Tipo retenciones'] == "IA") & (df_base_retencion['Ret'] == "SR")), 'Ret'] = "05"
    df_base_retencion.loc[((df_base_retencion['Tipo retenciones'] == "IA") & (df_base_retencion['Ret'] == "RB")), 'Ret'] = "35"
    df_base_retencion.loc[((df_base_retencion['Tipo retenciones'] == "IA") & (df_base_retencion['Ret'] == "RM")), 'Ret'] = "25"
    df_base_retencion.loc[((df_base_retencion['Tipo retenciones'] == "IA") & (df_base_retencion['Ret'] == "RA")), 'Ret'] = "15"
    
    
    #El dato maestro trae el tipo de riesgo segun padron
    # Prueba logica que determina el indicador en base al tipo de riesgo en la provincia de MENDOZA
    df_base_retencion.loc[((df_base_retencion['Tipo retenciones'] == "IM") & (df_base_retencion['Ret'] == "SR")), 'Ret'] = "AN"
    df_base_retencion.loc[((df_base_retencion['Tipo retenciones'] == "IM") & (df_base_retencion['Ret'] == "CR")), 'Ret'] = "BN"
    
    
    df_jurisdicciones_group = pd.merge(df_jurisdicciones_group, df_base_retencion, on='PROVINCIA', how='left')
    
    
    ###############################
    #Creamos un DF que contenga los impuestos con los que vamos a trabajar, y que asi se mantenga inalterable. Ya que no
    #todos los comprobantes poseen los mismos impuestos
    
    #1) Creamos un diccionario para definir el "IMPUESTO"
    #Posteriormente se podran modificar los valores
    cuadro_impuestos = {"IMPUESTO": ["IVA 10,5%", "PERCEPCION"]}


    #2) A partir del diccionario creamos un DataFrame
    df_cuadro_impuestos = pd.DataFrame(cuadro_impuestos)    
   
    
    df_cuadro_impuestos = pd.merge(df_cuadro_impuestos, df_impuestos_total, on='IMPUESTO', how='left')

    
    #Completamos los valores faltantes con cero
    df_cuadro_impuestos = df_cuadro_impuestos.fillna(0)
        

    #Obtenemos el valor correspondiente al IVA 10,5%    
    iva_total = df_cuadro_impuestos.iloc[0,1]
    
    
    ###############################
    ##########    IVA    ##########
    ###############################
    
    
    iva_porcentaje = 0.105
    
    
    ###############################
    ########## SUBTOTAL ###########
    ###############################    
    
    
    subtotal = (df_jurisdicciones_group['Total'].sum()).round(2)
    
    
    ###############################
    ######### PERCEPCION ##########
    ###############################    
    
    
    #Obtenemos el valor de la sumatoria de las percepciones
    percepcion_total = df_cuadro_impuestos.iloc[1,1]
    
    
    #Para obtener el porcentaje de la percepcion dividimos el valor de la percepcion total por el valor del subtotal
    #del comprobante
    percepcion_porcentaje = percepcion_total / df_jurisdicciones_group['Total'].sum()
      
    
    #Obtenemos el porcentaje de la percion y lo convertimos a texto para posteriormente detallarlo en el cuadro "Base Retenciones"
    percepcion_porcentaje_txt = str((percepcion_porcentaje).round(4)*100) + "%"
    
    
    ###############################
    ####### TOTAL FACTURA #########
    ###############################    
    
    
    total_factura = df_jurisdicciones_2['Total'].sum() + df_impuestos_3['Total'].sum()
    
    total_factura = total_factura.round(2)
    

    
    def calculo(row):
        if row['PROVINCIA'] == 'BUENOS AIRES':
            return row['Total'] * (1 + percepcion_porcentaje)
        
        if row['PROVINCIA'] == 'CAPITAL FEDERAL':
            return row['Total'] * (1 + percepcion_porcentaje)
        
        if row['PROVINCIA'] == 'CATAMARCA':
            return (row['Total'] * (1 + percepcion_porcentaje)) * 0.5        
        
        if row['PROVINCIA'] == 'CHACO':
            return row['Total'] * (1 + percepcion_porcentaje)        
        
        if row['PROVINCIA'] == 'CORDOBA':
            return ((row['Total'] * ( 1 + percepcion_porcentaje)) + (row['Imp. Gravado'] * iva_porcentaje)) * 0.8
        
        if row['PROVINCIA'] == 'CORRIENTES':
            return ((row['Total'] * ( 1 + percepcion_porcentaje)) + (row['Imp. Gravado'] * iva_porcentaje)) * 0.5
        
        if row['PROVINCIA'] == 'ENTRE RIOS':
            return row['Total'] * 0.5
        
        if row['PROVINCIA'] == 'FORMOSA':
            return row['Total'] * (1 + percepcion_porcentaje)        
        
        if row['PROVINCIA'] == 'JUJUY':
            return row['Total'] * (1 + percepcion_porcentaje)        
        
        if row['PROVINCIA'] == 'LA PAMPA':
            return row['Total'] * (1 + percepcion_porcentaje)
        
        if row['PROVINCIA'] == 'LA RIOJA':
            return row['Total'] * 0.5        
        
        if row['PROVINCIA'] == 'MENDOZA':
            return row['Total'] * (1 + percepcion_porcentaje)        
        
        if row['PROVINCIA'] == 'MISIONES':
            return row['Total'] * (1 + percepcion_porcentaje)        
        
        if row['PROVINCIA'] == 'RIO NEGRO':
            return row['Total'] * (1 + percepcion_porcentaje)        
        
        if row['PROVINCIA'] == 'SALTA':
            return (row['Total'] * (1 + percepcion_porcentaje)) * 0.5        
        
        if row['PROVINCIA'] == 'SAN LUIS':
            return (row['Total'] * (1 + percepcion_porcentaje)) * 0.5
        
        if row['PROVINCIA'] == 'SANTA CRUZ':
            return row['Total'] * 0.5        
        
        if row['PROVINCIA'] == 'SANTA FE':
            return ((row['Total'] * ( 1 + percepcion_porcentaje)) + (row['Imp. Gravado'] * iva_porcentaje))
        
        if row['PROVINCIA'] == 'SANTIAGO DEL ESTERO':
            return row['Total'] * (1 + percepcion_porcentaje)        
        
        if row['PROVINCIA'] == 'TIERRA DEL FUEGO':
            return ((row['Total'] * ( 1 + percepcion_porcentaje)) + (row['Imp. Gravado'] * iva_porcentaje))        
        
        if row['PROVINCIA'] == 'TUCUMAN':
            return row['Total'] * (1 + percepcion_porcentaje)   
        
        
    ###############################
    ###### Adic. especiales #######
    ###############################
    
    
    def calculo_2(row):
        if row['PROVINCIA'] == 'CORDOBA':
            return row['Total'] * (1 + percepcion_porcentaje)
        if row['PROVINCIA'] == 'TIERRA DEL FUEGO':
            if row['Total'] > 5000:
                return ((row['Total'] * ( 1 + percepcion_porcentaje)) + (row['Imp. Gravado'] * iva_porcentaje))
            
            
            
    def calculo_3(row):
        if row['PROVINCIA'] == 'CORDOBA':
            return 'MUNICIPALIDAD CORDOBA'
        if row['PROVINCIA'] == 'TIERRA DEL FUEGO':
            if row['Total'] > 5000:
                return 'ADIC. TIERRA DEL FUEGO'  
        
    
    
    #Creamos una copia del DF para lo que posteriormnete vamos a utilizar para el calculo de las NC/ND
    df_jurisdicciones_group_nc = df_jurisdicciones_group.copy()    
    
       
    #Definida la funcion la aplicamos sobre el DF, para obtener la columna 'CALCULO'
    df_jurisdicciones_group['CALCULO'] = df_jurisdicciones_group.apply(calculo, axis=1)
    
    
    #Definida la funcion la aplicamos sobre el DF, para obtener la columna 'CALCULO_2'
    df_jurisdicciones_group['CALCULO_2'] = df_jurisdicciones_group.apply(calculo_2, axis=1)
    
    
    #Definida la funcion la aplicamos sobre el DF, para obtener la columna 'CALCULO_3'
    df_jurisdicciones_group['CALCULO_3'] = df_jurisdicciones_group.apply(calculo_3, axis=1)    
    
    
    #Vamos a renombrar la columna 'CALCULO_2' para poder utilizarla como "BASE RETENCION"
    #Creamos una copia del DF
    base_retenciones_2 = df_jurisdicciones_group[['CALCULO_2', 'CALCULO_3']].copy()
    
        
    #Creamos un diccionario para definir el "TIPO RETENCION" y el "TIPO INDICADOR" para las jurisdicciones especiales
    #Posteriormente se podran modificar los valores
    retencion_especiales = {
        "Imp. Gravado": ["", ""],
        "Imp. Exento": ["", ""],
        "Total": ["", ""],
        "CALCULO_3": ["MUNICIPALIDAD CORDOBA", "ADIC. TIERRA DEL FUEGO"],
        "Tipo retenciones": ["0X", "TG"],
        "Ret": ["01", "01"],}
    
    
    #Lo convertimos a DF
    df_retencion_especiales = pd.DataFrame(retencion_especiales)    
    
    
    #Hacemos una union entre ambos DF´s para obtener el tipo y el indicador de retencion
    df_cuadro_impuestos_especiales = pd.merge(df_retencion_especiales, base_retenciones_2, on='CALCULO_3', how='left')

    
    #Filtramos la tabla "df_cuadro_impuestos_especiales" para excluir los NaN y asi obtener los montos distintos de cero
    #NaN no es igual a nada, por eso no se puede filtrar como texto. Se utiliza para ello isnull (negado)
    df_cuadro_impuestos_especiales = df_cuadro_impuestos_especiales.loc[~pd.isnull(df_cuadro_impuestos_especiales['CALCULO_2'])]

    
    #Cambiamos el nombre de la columna 'CALCULO_3' y 'CALCULO_2'
    df_cuadro_impuestos_especiales = df_cuadro_impuestos_especiales.rename(columns={'CALCULO_3':'PROVINCIA',
                                                                                    'CALCULO_2': 'BASE RETENCION'})
    
    
    #Cambiamos el nombre de la columna 'CALCULO'
    df_jurisdicciones_group = df_jurisdicciones_group.rename(columns={'CALCULO':'BASE RETENCION'})
    
    
    #Reordenamos las columnas segun la conveniencia para mostrar la informacion y excluimos las que no necesitamos
    df_jurisdicciones_group = df_jurisdicciones_group.reindex(['Imp. Gravado', 'Imp. Exento', 'Total', 'PROVINCIA',
                                                               'Tipo retenciones', 'Ret', 'BASE RETENCION'], axis=1)
    
    
    #Concatenamos amobs DF's en forma vertical (deben coincidir todos los titulos de las columnas)
    df_jurisdicciones_group = pd.concat([df_jurisdicciones_group, df_cuadro_impuestos_especiales], axis =0)
    
    
    df_cantidad = len(df)


    #Creamos el rango para el formato condicional
    df_rango = "A" + str(df_cantidad+4) + ":" + "E" + str(df_cantidad+4)



    
    
    #Le damos el mismo formato que tiene en SAP
    nombre_hoja = (nombre_hoja).replace('N°','0')


    nombre_hoja = (nombre_hoja).replace('-','A')
    
    
    #Cantidad de filas del cuadro "df_jurisdicciones_group" en la pestaña "Base Retenciones"
    filas = len(df_jurisdicciones_group)
    
    
    #Creamos el DF definitivo que nos permitira copiar las celdas de Excel y pegarlas en SAP
    #Primero deberemos construir el diccionario que contenga los campos a utilizar
    cuadro_sap = {
        "Tipo retenciones": ["0X", "CP", "I1", "IA", "IB", "IC", "ID", "IF", "IG", "IH", "II", "IK",
                             "IL", "IM", "IN", "IO", "IT", "IV", "IW", "IX", "IY", "SA", "SD", "TF", "TG"]}

    
    #A partir del diccionario creamos un DataFrame
    df_cuadro_sap = pd.DataFrame(cuadro_sap)
    
            
    df_cuadro_sap = pd.merge(df_cuadro_sap, df_jurisdicciones_group[['Tipo retenciones', 'Ret', 'BASE RETENCION']],
                                                                     on="Tipo retenciones", how='left')
    
    
    #Creamos un diccionario para luego incorporar ganancias al cuadro df_jurisdicciones_group    
    cuadro_ganancias = {"Tipo retenciones": ["GA"], "Ret": ['07'], "BASE RETENCION": [subtotal]}
    
    
    #A partir del diccionario creamos un DataFrame
    df_cuadro_ganancias = pd.DataFrame(cuadro_ganancias)
    
    
    #Concatenamos ambos DF's en forma vertical (deben coincidir todos los titulos de las columnas)
    df_cuadro_sap = pd.concat([df_cuadro_sap, df_cuadro_ganancias], axis =0)
    
    
    #Ordenamos el DF de acuerdo a "Tipo retenciones", para asignar GA al lugar que le corresponde
    df_cuadro_sap = df_cuadro_sap.sort_values(by=["Tipo retenciones"])
    

    
    #Validamos que aquellos comprobantes que tengan la columna "BASE RETENCION" en cero; incorporen la leyenda "sin detalle"
    #Para eso sumamos la columna "BASE RETENCION"
    if df_jurisdicciones_group['BASE RETENCION'].sum() == 0:
        
        iibb = {"COMPROBANTE": [nombre_hoja],
                "IIBB": ["SIN DETALLE"]}
        
        
    else:
        
         iibb = {"COMPROBANTE": [nombre_hoja],
                "IIBB": [""]}       
    
    
    #Convertimos el diccionario a un DataFrame
    iibb = pd.DataFrame(iibb)
    

    
    ######################################
    ########## Notas de Credito ##########
    ######################################



    #Prueba logica que valida si el numero de factura se encuentre en el DataFrame "asociados"
    if nombre_hoja in asociados.FACTURA.values:
        
        #Extraemos el numero de indice en el que se encuentra la factura que buscamos
        indice = asociados.index[asociados['FACTURA'] == nombre_hoja].tolist()[0]

        #Extraemos el numero de factura (si existe en la tabla "asociados")
        num_fact = asociados.loc[indice, 'FACTURA']


        #Agregamos .values[0] para obtener solo el valor de la Serie
        total_nc = asociados.loc[asociados['FACTURA'] == nombre_hoja,'Total'].values[0]
        
        
        #Obtenemos el "coeficiente", hacer el total de la nc dividido el total de la factura
        coeficiente = (total_nc / subtotal)

        def calculo_nc(row):
            if row['PROVINCIA'] == 'BUENOS AIRES':
                return row['Total'] * coeficiente

            if row['PROVINCIA'] == 'CAPITAL FEDERAL':
                return row['Total'] * coeficiente

            if row['PROVINCIA'] == 'CATAMARCA':
                return row['Total'] * 0.5 * coeficiente       

            if row['PROVINCIA'] == 'CHACO':
                return row['Total'] * coeficiente    

            if row['PROVINCIA'] == 'CORDOBA':
                return row['Total'] * 0.8 * coeficiente

            if row['PROVINCIA'] == 'CORRIENTES':
                return row['Total'] * 0.5 * coeficiente

            if row['PROVINCIA'] == 'ENTRE RIOS':
                return row['Total'] * 0.5 * coeficiente

            if row['PROVINCIA'] == 'FORMOSA':
                return row['Total'] * coeficiente        

            if row['PROVINCIA'] == 'JUJUY':
                return row['Total'] * coeficiente        

            if row['PROVINCIA'] == 'LA PAMPA':
                return row['Total'] * coeficiente

            if row['PROVINCIA'] == 'LA RIOJA':
                return row['Total'] * 0.5 * coeficiente       

            if row['PROVINCIA'] == 'MENDOZA':
                return row['Total'] * coeficiente        

            if row['PROVINCIA'] == 'MISIONES':
                return row['Total'] * coeficiente        

            if row['PROVINCIA'] == 'RIO NEGRO':
                return row['Total'] * coeficiente        

            if row['PROVINCIA'] == 'SALTA':
                return row['Total'] * 0.5 * coeficiente        

            if row['PROVINCIA'] == 'SAN LUIS':
                return row['Total'] * 0.5 * coeficiente

            if row['PROVINCIA'] == 'SANTA CRUZ':
                return row['Total'] * 0.5 * coeficiente       

            if row['PROVINCIA'] == 'SANTA FE':
                return row['Total'] * coeficiente

            if row['PROVINCIA'] == 'SANTIAGO DEL ESTERO':
                return row['Total'] * coeficiente        

            if row['PROVINCIA'] == 'TIERRA DEL FUEGO':
                return row['Total'] * coeficiente        

            if row['PROVINCIA'] == 'TUCUMAN':
                return row['Total'] * coeficiente   


        ###############################
        ###### Adic. especiales #######
        ###############################


        def calculo_2_nc(row):
            if row['PROVINCIA'] == 'CORDOBA':
                return row['Total'] * coeficiente
            if row['PROVINCIA'] == 'TIERRA DEL FUEGO':
                if row['Total'] > 5000:
                    return row['Total'] * coeficiente



        def calculo_3_nc(row):
            if row['PROVINCIA'] == 'CORDOBA':
                return 'MUNICIPALIDAD CORDOBA'
            if row['PROVINCIA'] == 'TIERRA DEL FUEGO':
                if row['Total'] > 5000:
                    return 'ADIC. TIERRA DEL FUEGO'  




        #Definida la funcion la aplicamos sobre el DF, para obtener la columna 'CALCULO'                    
        df_jurisdicciones_group_nc['CALCULO'] = df_jurisdicciones_group_nc.apply(calculo_nc, axis=1)


        #Definida la funcion la aplicamos sobre el DF, para obtener la columna 'CALCULO_2'                      
        df_jurisdicciones_group_nc['CALCULO_2'] = df_jurisdicciones_group_nc.apply(calculo_2_nc, axis=1)


        #Definida la funcion la aplicamos sobre el DF, para obtener la columna 'CALCULO_3'
        df_jurisdicciones_group_nc['CALCULO_3'] = df_jurisdicciones_group_nc.apply(calculo_3_nc, axis=1)    


        #Vamos a renombrar la columna 'CALCULO_2' para poder utilizarla como "BASE RETENCION"
        #Creamos una copia del DF
        base_retenciones_2_nc = df_jurisdicciones_group_nc[['CALCULO_2', 'CALCULO_3']].copy()


        #Creamos un diccionario para definir el "TIPO RETENCION" y el "TIPO INDICADOR" para las jurisdicciones especiales
        #Posteriormente se podran modificar los valores
        retencion_especiales_nc = {
            "Imp. Gravado": ["", ""],
            "Imp. Exento": ["", ""],
            "Total": ["", ""],
            "CALCULO_3": ["MUNICIPALIDAD CORDOBA", "ADIC. TIERRA DEL FUEGO"],
            "Tipo retenciones": ["0X", "TG"],
            "Ret": ["01", "01"],}


        #Lo convertimos a DF
        df_retencion_especiales_nc = pd.DataFrame(retencion_especiales_nc)    


        #Hacemos una union entre ambos DF´s para obtener el tipo y el indicador de retencion
        df_cuadro_impuestos_especiales_nc = pd.merge(df_retencion_especiales_nc, base_retenciones_2_nc, on='CALCULO_3', how='left')


        #Filtramos la tabla "df_cuadro_impuestos_especiales" para excluir los NaN y asi obtener los montos distintos de cero
        #NaN no es igual a nada, por eso no se puede filtrar como texto. Se utiliza para ello isnull (negado)
        df_cuadro_impuestos_especiales_nc = df_cuadro_impuestos_especiales_nc.loc[~pd.isnull(df_cuadro_impuestos_especiales_nc['CALCULO_2'])]


        #Cambiamos el nombre de la columna 'CALCULO_3' y 'CALCULO_2'
        df_cuadro_impuestos_especiales_nc = df_cuadro_impuestos_especiales_nc.rename(columns={'CALCULO_3':'PROVINCIA',
                                                                                              'CALCULO_2': 'BASE RETENCION'})


        #Cambiamos el nombre de la columna 'CALCULO'
        df_jurisdicciones_group_nc = df_jurisdicciones_group_nc.rename(columns={'CALCULO':'BASE RETENCION'})


        #Reordenamos las columnas segun la conveniencia para mostrar la informacion y excluimos las que no necesitamos
        df_jurisdicciones_group_nc = df_jurisdicciones_group_nc.reindex(['Imp. Gravado', 'Imp. Exento', 'Total', 'PROVINCIA',
                                                                         'Tipo retenciones', 'Ret', 'BASE RETENCION'], axis=1)


        #Concatenamos amobs DF's en forma vertical (deben coincidir todos los titulos de las columnas)
        df_jurisdicciones_group_nc = pd.concat([df_jurisdicciones_group_nc, df_cuadro_impuestos_especiales_nc], axis =0)


        # #Creamos el DF definitivo que nos permitira copiar las celdas de Excel y pegarlas en SAP
        # #Primero deberemos construir el diccionario que contenga los campos a utilizar
        cuadro_sap_nc = {"Tipo retenciones": ["0X", "CP", "I1", "IA", "IB", "IC", "ID", "IF", "IG", "IH", "II", "IK",
                                               "IL", "IM", "IN", "IO", "IT", "IV", "IW", "IX", "IY", "SA", "SD", "TF", "TG"]}


        #A partir del diccionario creamos un DataFrame
        df_cuadro_sap_nc = pd.DataFrame(cuadro_sap_nc)


        df_cuadro_sap_nc = pd.merge(df_cuadro_sap_nc, df_jurisdicciones_group_nc[['Tipo retenciones', 'Ret', 'BASE RETENCION']],                            
                                                                              on="Tipo retenciones", how='left')


        #Creamos un diccionario para luego incorporar ganancias al cuadro df_jurisdicciones_group    
        cuadro_ganancias_nc = {"Tipo retenciones": ["GA"], "Ret": ['07'], "BASE RETENCION": [total_nc]}


        #A partir del diccionario creamos un DataFrame
        df_cuadro_ganancias_nc = pd.DataFrame(cuadro_ganancias_nc)


        #Concatenamos ambos DF's en forma vertical (deben coincidir todos los titulos de las columnas)
        df_cuadro_sap_nc = pd.concat([df_cuadro_sap_nc, df_cuadro_ganancias_nc], axis =0)


        #Ordenamos el DF de acuerdo a "Tipo retenciones", para asignar GA al lugar que le corresponde
        df_cuadro_sap_nc = df_cuadro_sap_nc.sort_values(by=["Tipo retenciones"])
        
          
        #Extraemos el numero de comprobante
        num_nc_nd = asociados.loc[indice, 'NC/ND']
        
        
          
    else:
        
        #Creamos un diccionario sin datos
        df_cuadro_sap_nc = {"Tipo ret": [],
                            "Ret": [],
                            "BASE RETENCION": []}
        
        
        #A partir del diccionario creamos un DataFrame sin datos
        #Xlsxwriter deshabilita los titulos. Por ese motivo, el Dataframe, aparenta estar en blanco cuando se transcribe a Excel
        df_cuadro_sap_nc = pd.DataFrame(df_cuadro_sap_nc)
        
        
        #Creamos un numero de comprobante asociado en blanco
        num_nc_nd = ""
    
    
    
    ################################
    ########## XLSXWRITER ##########
    ################################


    with pd.ExcelWriter(nombre_hoja + ".xlsx") as writer:
        df_jurisdicciones_2.to_excel(writer, index = False, sheet_name = nombre_hoja)
        df_jurisdicciones_3.to_excel(writer, index = False, sheet_name = nombre_hoja, startcol = 6)
        df_subtotal_general.to_excel(writer, index = False,  sheet_name = nombre_hoja, startrow = cantidad_lineas + 2, header=None)
        df_impuestos_2.to_excel(writer, index = False,  sheet_name = nombre_hoja, startrow = cantidad_lineas + 4, header=None)
        df_impuestos_3.to_excel(writer, index = False,  sheet_name = nombre_hoja, startrow = cantidad_lineas + 4, startcol = 4, header=None)
        df_clase_impuesto_2.to_excel(writer, index = False,  sheet_name = nombre_hoja, startrow = cantidad_lineas + 4, startcol = 6, header=None)
        df_total.to_excel(writer, index = False,  sheet_name = nombre_hoja, startrow = df_cantidad + 4, startcol = 4, header=None)
        df_jurisdicciones_group.to_excel(writer, index = False, sheet_name="Base Retenciones", startrow = 5)
        df_cuadro_impuestos.to_excel(writer, index = False, sheet_name="Base Retenciones", startrow = 2, header=None)
        df_cuadro_sap.to_excel(writer, index = False, sheet_name="Base Retenciones", startrow = 6, startcol = 10, header=None)
        df_cuadro_sap_nc.to_excel(writer, index = False, sheet_name="Base Retenciones", startrow = 6, startcol = 14, header=None)


        
        workbook1 = writer.book

        worksheet1 = writer.sheets[nombre_hoja]
    #Now we have the worksheet object. We can manipulate it

    #Defino el formato en Excel para el Dataframe:


        #_Color de las celdas a utilizar en "Comprobante"   
        header_format = workbook1.add_format({
            "border": 2, #borde
            "border_color": "#FFFFFF", #color de borde
            "valign": "vcenter", #centra el texto en vertical
            "align": "center", #centra el texto en horizontal
            "bg_color": "#4BACC6",
            "bold": True,
            "font_color": "#FFFFFF" #color de la fuente
        })



        #_Color de las celdas a utilizar en "DETALLE"   
        header_format_0 = workbook1.add_format({
            "border": 2, #borde
            "border_color": "#FFFFFF", #color de borde            
            "valign": "vcenter", #centra el texto en vertical
            "align": "center", #centra el texto en horizontal        
            "bg_color": "#DDD9C4",
            "bold": True})  



        #_Color de las celdas a utilizar en contorno exterior superior   
        header_format_1 = workbook1.add_format({
            "top": 2,
            "top_color": "#4BACC6"})


        #_Color de las celdas a utilizar en contorno exterior vertical   
        header_format_2 = workbook1.add_format({
            "left": 2,
            "left_color": "#4BACC6"})


        #_Color de las celdas a utilizar en contorno interior vertical    
        header_format_3 = workbook1.add_format({        
            "right": 1,
            "right_color": "#A6A6A6"})



        #_Formato numero con 2 decimales y separador de miles    
        header_format_4 = workbook1.add_format({
            'num_format': '#,##0.00'})         


        
        #_Color de las celdas a utilizar en contorno interior vertical    
        header_format_5 = workbook1.add_format({        
            "border": 2, #borde
            "border_color": "#FFFFFF", #color de borde            
            "valign": "vcenter", #centra el texto en vertical
            "align": "center", #centra el texto en horizontal        
            "bg_color": "#EEECE1",
            "bold": True,
            'num_format': '#,##0.00'})  
      

    
    
        #Color condicional de las celdas que contengan CHUBUT, NEUQUEN, SAN JUAN  
        header_format_6 = workbook1.add_format({        
            "bg_color": "#FFC7CE",
            "bold": True,
            "font_color": "#9C0006"}) #color de la fuente
        
        
        
        #_Color de las celdas a utilizar en titulo "SAP"   
        header_format_7 = workbook1.add_format({
            "border": 2, #borde
            "border_color": "#FFFFFF", #color de borde
            "valign": "vcenter", #centra el texto en vertical
            "align": "center", #centra el texto en horizontal
            "bg_color": "#4BACC6",
            "bold": True,
            "font_color": "#FFFFFF", #color de la fuente
            "font_size": 14,
            "font_name": "72 Black"})
        
        
        
        #_Color de las celdas a utilizar en subtitulos "SAP"   
        header_format_8 = workbook1.add_format({
            "border": 2, #borde
            "border_color": "#FFFFFF", #color de borde
            "valign": "vcenter", #centra el texto en vertical
            "align": "center", #centra el texto en horizontal
            "bg_color": "#8DB4E2",
            "bold": True,
            "font_color": "#FFFFFF" #color de la fuente
        })
        
        
        
        #_Color de las celdas a utilizar en titulo "CREDITO"   
        header_format_credito = workbook1.add_format({
            "border": 2, #borde
            "border_color": "#FFFFFF", #color de borde
            "valign": "vcenter", #centra el texto en vertical
            "align": "center", #centra el texto en horizontal
            "bg_color": "#DA9694",
            "bold": True,
            "font_color": "#FFFFFF", #color de la fuente
            "font_size": 14,
            "font_name": "72 Black"})
        
        
        
        ################################
        ######## Comprobante N° ########
        ################################        


        #_Renombra las columnas
        worksheet1.write('A1', 'ITEM', header_format)
        worksheet1.write('B1', 'DETALLE', header_format)
        worksheet1.write('C1', 'Imp. Gravado', header_format)
        worksheet1.write('D1', 'Imp. Exento', header_format)
        worksheet1.write('E1', 'Total', header_format)
        worksheet1.write('F1', '')
        worksheet1.write('G1', 'DETALLE', header_format_0)
        worksheet1.write('D' + str(df_cantidad + 5), 'TOTAL', header_format)


        #_Ancho de las columnas
        worksheet1.set_column('A:A', 6)
        worksheet1.set_column('B:B', 63)
        worksheet1.set_column('C:C', 14, header_format_4)
        worksheet1.set_column('D:D', 14, header_format_4)
        worksheet1.set_column('E:E', 14, header_format_4)
        worksheet1.set_column('F:F', 10)
        worksheet1.set_column('G:G', 21)


        #_Alto de la fila
        worksheet1.set_row(0, 20)


        #Formato condicional que convierte a blanco el texto de la celda A8, cuando no haya diferencia
        #Para comparar texto se debe utilizar comilla doble y comilla simple por fuera de la formula   
        worksheet1.conditional_format(df_rango, {"type": "formula", "criteria": '=($A$1 = "ITEM")', "format": header_format_1})
        worksheet1.conditional_format("A2:" + "D" + str(df_cantidad+3), {"type": "formula", "criteria": '=($A$1 = "ITEM")', "format": header_format_3})
        worksheet1.conditional_format("F2:" + "F" + str(df_cantidad+3), {"type": "formula", "criteria": '=($A$1 = "ITEM")', "format": header_format_2})


        #Porcentaje visualizacion de la hoja
        worksheet1.set_zoom(90)




        ######################################
        ########## Base Retenciones ##########
        ######################################


        worksheet2 = writer.sheets["Base Retenciones"]
        #Now we have the worksheet object. We can manipulate it

        #Defino el formato en Excel para el Dataframe:


        #_Renombra las columnas
        worksheet2.write('A1', 'COMPROBANTE', header_format)
        worksheet2.write('B1', nombre_hoja, header_format_5)
        worksheet2.write('A3', 'IVA 10,5%', header_format)
        worksheet2.write('A4', 'PERCEPCION ' + percepcion_porcentaje_txt, header_format)
        worksheet2.write('B3', df_cuadro_impuestos['Total'][0], header_format_5)
        worksheet2.write('B4', df_cuadro_impuestos['Total'][1], header_format_5)
        worksheet2.write('A6', 'Imp. Gravado', header_format)
        worksheet2.write('B6', 'Imp. Exento', header_format)
        worksheet2.write('C6', 'Total', header_format)
        worksheet2.write('D6', 'PROVINCIA', header_format)
        worksheet2.write('E6', 'Tipo retenciones', header_format_0)
        worksheet2.write('F6', 'Ret', header_format_0)
        worksheet2.write('G6', 'BASE RETENCION', header_format_0)
        worksheet2.write('K6', 'Tipo ret', header_format_8)
        worksheet2.write('L6', 'Ret', header_format_8)        
        worksheet2.write('M6', 'BASE RETENCION', header_format_8)
        worksheet2.write('O6', 'Tipo ret', header_format_8)
        worksheet2.write('P6', 'Ret', header_format_8)        
        worksheet2.write('Q6', 'BASE RETENCION', header_format_8)
        
        
        
        #Combina celdas y aplica formato (SAP)
        worksheet2.merge_range('K5:M5', 'SAP', header_format_7)
        
        
        #Combina celdas y aplica formato (SAP)
        worksheet2.merge_range('O5:Q5', 'SAP', header_format_7)
        
        
        #Combina celdas y aplica formato (SAP). Las identifica con el numero de comprobante asociado
        worksheet2.merge_range('O4:Q4', num_nc_nd, header_format_credito)
        
        
        
        #_Alto de la fila (SAP)
        worksheet2.set_row(4, 18.75)
        worksheet2.set_row(3, 18.75)
        
        

        #_Ancho de las columnas
        worksheet2.set_column('A:C', 21.4, header_format_4)
        worksheet2.set_column('D:D', 24.3)
        worksheet2.set_column('E:E', 16)
        worksheet2.set_column('F:F', 3.5)
        worksheet2.set_column('G:G', 16, header_format_4)
        worksheet2.set_column('M:M', 18, header_format_4)
        worksheet2.set_column('Q:Q', 18, header_format_4)
       
    
        rango = "D7:D" + str(filas + 7) 
        
        #Referenciamos la columna D como absoluta, sin embargo la fila es relativa.
        #Esto permite que a medida que nos desplacemos por el rango fijado, solo se modifique el valor de la fila
        #mientras que la columna permanece inalterable.
        worksheet2.conditional_format(rango, {'type': 'formula',
                                          'criteria': '=OR($D7="CHUBUT", $D7="NEUQUEN", $D7="SAN JUAN")',
                                          'format': header_format_6})    


        
        #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
        #Se aplica sobre el cuadro SAP
        worksheet2.conditional_format('K7:K32', {'type' : 'cell',
                                     'criteria' : 'not equal to', 
                                     'value' : '"Error"', 
                                     'format': header_format_2,
                                     'multi_range': 'K7:K32 N7:N32'}) #Extiende el rango sobre el cual aplica el formato
        
        
        
        #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
        #Se aplica sobre el cuadro SAP
        worksheet2.conditional_format('K33:M33', {'type' : 'cell',
                                     'criteria' : 'not equal to', 
                                     'value' : '"Error"', 
                                     'format': header_format_1})
        
        
        
        #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
        #Se aplica sobre el cuadro CREDITO
        worksheet2.conditional_format('O7:O32', {'type' : 'formula',
                                     'criteria' : '=$O$9="GA"', 
                                     'format': header_format_2,
                                     'multi_range': 'O7:O32 R7:R32'}) #Extiende el rango sobre el cual aplica el formato
        
        
        
        #Se utiliza el formato condicional, dado que nos permite aplicar el formato sobre un rango especifico
        #Se aplica sobre el cuadro CREDITO. Aplicamos el tipo formula, ya que de esta forma nos permite referenciar
        #la condicion a una celda especifica y aplicarla sobre un rango.
        worksheet2.conditional_format('O33:Q33', {'type' : 'formula',
                                     'criteria' : '=$O$9="GA"', 
                                     'format': header_format_1})        
        
        
        
        #Oculta las lineas de cuadricula de la hoja
        worksheet2.hide_gridlines([1])
        
        
        #Porcentaje visualizacion de la hoja
        worksheet2.set_zoom(90)
                
        
        
        
######################################
########## Total Consolidado #########
######################################



    #Concatenamos los DF's de FC/NC/ND en forma vertical (deben coincidir todos los titulos de las columnas)
    #Esto nos permitira, posteriormente, sumar los comprobantes NC/ND con FC
    df_cuadro_sap = pd.concat([df_cuadro_sap, df_cuadro_sap_nc], axis =0)
    
    
    #A medida que se ejecuta el ciclo "for" va agregando los DF´s a la lista vacia que creamos al inicio
    dfs_consolidado_retenciones.append(df_cuadro_sap)
    
    #A medida que se ejecuta el ciclo "for" va agregando los comprobantes a la lista vacia que creamos al inicio
    dfs_consolidado_comprobantes.append(nombre_hoja)
    
    #A medida que se ejecuta el ciclo "for" va agregando el total, de cada comprobante, a la lista vacia que creamos al inicio
    dfs_consolidado_totales.append(total_factura)
    
    #A medida que se ejecuta el ciclo "for" va agregando el total leido de cada .PDF
    dfs_consolidado_totales_pdf.append(df_total)
       
    #A medida que se ejecuta el ciclo "for" va agregando los DF´s a la lista vacia que creamos al inicio
    dfs_consolidado_iibb.append(iibb)
    


#Finalizado el ciclo "for"
#Concatenamos todos los DF's en uno solo
#"ignore_index" para reindexar las filas
final_df = pd.concat(dfs_consolidado_retenciones, ignore_index=True)


#Finalizado el ciclo "for"
#Concatenamos todos los DF's en uno solo
#"ignore_index" para reindexar las filas
final_dfs_consolidado_iibb = pd.concat(dfs_consolidado_iibb, ignore_index=True)


#Al aplicar groupby totalizamos cada uno de los impuestos segun su tipo y totalizamos segun su 'BASE RETENCION'
#Excluye CHUBUT, NEUQUEN, SAN JUAN por tener NaN entre sus valores 
final_df = final_df.groupby(['Tipo retenciones', 'Ret']).agg({'BASE RETENCION': 'sum'}).reset_index()


#Creamos el DF definitivo que nos permitira copiar las celdas de Excel y pegarlas en SAP
#Primero deberemos construir el diccionario que contenga los campos a utilizar
final_ret = {
    "Tipo retenciones": ["0X", "CP", "GA", "I1", "IA", "IB", "IC", "ID", "IF", "IG", "IH", "II", "IK",
                         "IL", "IM", "IN", "IO", "IT", "IV", "IW", "IX", "IY", "SA", "SD", "TF", "TG"]}

    
#A partir del diccionario creamos un DataFrame
df_final_ret = pd.DataFrame(final_ret)


#Hacemos una union entre ambos DF´s para obtener el tipo y el indicador de retencion
final_df_ret = pd.merge(df_final_ret, final_df, on='Tipo retenciones', how='left')


#Creamos un diccionario que contenga la lista de comprobantes
final_df_comprobantes = {"COMPROBANTE": dfs_consolidado_comprobantes}


#Lo convertimos a DF
final_df_comprobantes = pd.DataFrame(final_df_comprobantes)


#Creamos un diccionario que contenga la lista de monto total calculado por comprobante
final_df_totales = {"total": dfs_consolidado_totales}


#Lo convertimos a DF
final_df_totales = pd.DataFrame(final_df_totales)


#Finalizado el ciclo "for" concatenamos todos los totales leidos en un DF
final_df_totales_pdf = pd.concat(dfs_consolidado_totales_pdf, ignore_index=True)


#Concatenamos los DF's para realizar una verificacion final
final_df_consolidado = pd.concat([final_df_comprobantes, final_df_totales, final_df_totales_pdf], axis = 1)


#Verificamos que el "total calculado" coincida con el "total leido" de la factura
final_df_consolidado['DIF'] = final_df_consolidado['total'] - final_df_consolidado['TOTAL']


#Creamos un diccionario con los datos de los totales de cada columna
total_fac_pdf_dif = {
        "COMPROBANTE": [""],
        "total": [""],
        "TOTAL": [final_df_consolidado['TOTAL'].sum()],
        "DIF": [""]}


#Lo convertimos a DF
total_fac_pdf_dif = pd.DataFrame(total_fac_pdf_dif)


#Concatenamos los DF's para realizar una verificacion final de los totales y comprobar si la diferencia es cero
final_df_consolidado = pd.concat([final_df_consolidado, total_fac_pdf_dif], axis = 0)


final_df_consolidado = pd.merge(final_df_consolidado, final_dfs_consolidado_iibb, on='COMPROBANTE', how='left')


# Prueba logica que inserta la palabra "COMPROBANTE" cuando detecta que en la columna adyacente esta la palabra 'SIN DETALLE' 
final_df_consolidado.loc[(final_df_consolidado['IIBB'] == "SIN DETALLE"), 'DIF'] = "COMPROBANTE"


#De las dimensiones del DF "final_df_consolidado" tomamos la cantidad de filas 
filas_df_final = final_df_consolidado.shape[0]


#Creamos el rango para el titulo de "CONTROL NC/ND"
rango_control = "H" + str(filas_df_final + 6) + ":K" + str(filas_df_final + 6)


#Creamos un DF que contenga los COMPROBANTES y las NC/ND a las cuales estan referenciados
#Primero renombramos la columna 'COMPROBANTE' por 'FACTURA'
final_df_comprobantes = final_df_comprobantes.rename(columns={'COMPROBANTE':'FACTURA'})


#La columna REFERENCIA la dejamos en blanco. De esta forma cuando hagamos el cruce de DF solo se informaran los NaN
final_df_comprobantes['REFERENCIA'] = ""


#Realizamos un merge para verificar los comprobantes que no se encontraron en la carpeta
df_comprobantes_ncnd = pd.merge(asociados, final_df_comprobantes, on='FACTURA', how='left')


#Indicamos que se completen los valores NaN, de la columna REFERENCIA, con la palabra 'COMPROBANTE'
df_comprobantes_ncnd['REFERENCIA'].fillna('COMPROBANTE', inplace=True)


# Prueba logica que inserta la palabra "INEXISTENTE" cuando detecta que en la columna adyacente esta la palabra 'COMPROBANTE' 
df_comprobantes_ncnd.loc[(df_comprobantes_ncnd['REFERENCIA'] == "COMPROBANTE"), 'Ret'] = "INEXISTENTE"













#Creamos un diccionario con el total de la columna correspondiente; para luego concatenarlo a NC/ND
total_ncnd = {
        "FACTURA": ["TOTAL"],
        "NC/ND": [""],
        "Total": [df_comprobantes_ncnd['Total'].sum()],
        "REFERENCIA": [""],
        "Ret": [""]}


#Lo convertimos a DF
total_ncnd = pd.DataFrame(total_ncnd)


#Concatenamos los DF's para obtener el total de las NC/ND
df_comprobantes_ncnd = pd.concat([df_comprobantes_ncnd, total_ncnd], axis = 0)



######################################
############# Xlsxwriter #############
######################################



with pd.ExcelWriter("Total Consolidado" + ".xlsx") as writer:
    final_df_ret.to_excel(writer, index = False, sheet_name = "Total Consolidado", startrow = 2, startcol = 1)
    final_df_consolidado.to_excel(writer, index = False, sheet_name = "Total Consolidado", startrow = 2, startcol = 7)
    df_comprobantes_ncnd.to_excel(writer, index = False, sheet_name = "Total Consolidado", startrow = filas_df_final + 7, startcol = 7, header=None)
    
    
    
    workbook3 = writer.book

    worksheet3 = writer.sheets["Total Consolidado"]
    #Now we have the worksheet object. We can manipulate it

    #Defino el formato en Excel para el Dataframe:
    
    
    #_Color de las celdas a utilizar en "Comprobante"   
    header_format_w3 = workbook3.add_format({
            "border": 2, #borde
            "border_color": "#FFFFFF", #color de borde
            "valign": "vcenter", #centra el texto en vertical
            "align": "center", #centra el texto en horizontal
            "bg_color": "#8DB4E2",
            "bold": True,
            "font_color": "#FFFFFF" #color de la fuente
     })
    

    
    #El tamaño de la fuente se asigna a la columna, ya que el formato condicional no lo aplica
    header_format_pro_columna = workbook3.add_format({'font_size': 10})
    
    
    
    #_Formato numero con 2 decimales y separador de miles    
    header_format_4 = workbook3.add_format({'num_format': '#,##0.00'})
    
    
    
    #_Color de las celdas a utilizar en "COMPROBANTE", "TOTAL FAC", etc    
    header_format_0 = workbook3.add_format({
        "border": 2, #borde
        "border_color": "#FFFFFF", #color de borde            
        "valign": "vcenter", #centra el texto en vertical
        "align": "center", #centra el texto en horizontal        
        "bg_color": "#DDD9C4",
        "bold": True}) 
    
    
    
    #_Color de los bordes a utilizar en el DF "final_df" (lineas intermedias)      
    header_format_pro_borde = workbook3.add_format({
        "bottom": 7, #borde
        "bottom_color": "#EEECE1" #color de borde            
        })
    
    
    
    header_format_total = workbook3.add_format({
        "bold": True})
    
    
    
    #_Color de las celdas a utilizar en titulo "SAP"   
    header_format_sap = workbook3.add_format({
        "border": 2, #borde
        "border_color": "#FFFFFF", #color de borde
        "valign": "vcenter", #centra el texto en vertical
        "align": "center", #centra el texto en horizontal
        "bg_color": "#4BACC6",
        "bold": True,
        "font_color": "#FFFFFF", #color de la fuente
        "font_size": 14,
        "font_name": "72 Black"})
    
    
    
    #_Color de las celdas a utilizar en titulo "SAP"   
    header_format_control = workbook3.add_format({
        "border": 2, #borde
        "border_color": "#FFFFFF", #color de borde
        "valign": "vcenter", #centra el texto en vertical
        "align": "center", #centra el texto en horizontal
        "bg_color": "#0D0D0D",
        "bold": True,
        "font_color": "#FFFFFF", #color de la fuente
        "font_size": 14,
        "font_name": "72 Black"})    
    
    
    
    #_Color de las celdas a utilizar en contorno exterior vertical del cuadro SAP   
    header_format_2 = workbook3.add_format({
        "left": 2,
        "left_color": "#4BACC6"})
    
    
    
    #_Color de las celdas a utilizar en contorno exterior vertical del cuadro SAP   
    header_format_3 = workbook3.add_format({
        "right": 2,
        "right_color": "#4BACC6"})
    
    
    
    #_Color de las celdas a utilizar en contorno exterior inferior del cuadro SAP      
    header_format_5 = workbook3.add_format({
        "top": 2, #borde
        "top_color": "#4BACC6" #color de borde            
        })

    
    
    #Color condicional de las celdas que contengan diferencia (disitnto a 0) en la columna K  
    header_format_6 = workbook3.add_format({        
        "bg_color": "#FFC7CE",
        "bold": True,
        "font_color": "#9C0006"}) #color de la fuente
    
    
    
    #Color condicional de las celdas que contengan la palabra "COMPROBANTE"  
    header_format_comp_inex = workbook3.add_format({        
        "bg_color": "#CCC0DA",
        "bold": True,
        "font_color": "#60497A"}) #color de la fuente
    
    
    
    #Color condicional de las celdas que contengan la palabra "INEXISTENTE"  
    header_format_inex = workbook3.add_format({        
        "bg_color": "#8064A2",
        "bold": True,
        "font_color": "#FFFFFF"}) #color de la fuente
    
   

    #Color condicional de las celdas que contengan diferencia (disitnto a 0) en la columna K  
    header_format_detalle = workbook3.add_format({        
        "bg_color": "#9C0006",
        "bold": True,
        "font_color": "#FFFFFF"}) #color de la fuente
    
    
    
    #_Color de las celdas a utilizar en "TOTAL"    
    header_format_tot = workbook3.add_format({
        "border": 2, #borde
        "border_color": "#FFFFFF", #color de borde            
        "valign": "vcenter", #centra el texto en vertical
        "align": "center", #centra el texto en horizontal        
        "bg_color": "#DDD9C4",
        "bold": True,
        "font_color": "#FFFFFF",
        "font_name": "72 Black"})    
    
    
      
    #Cantidad de filas del cuadro "final_df_consolidado"
    filas_comprobante = len(final_df_consolidado['COMPROBANTE'])
    
    celda_comprobante = "H" + str(filas_comprobante + 3)
    
    #Rango que contiene los totales de cada columna
    rango_comprobante = celda_comprobante + ":K" + str(filas_comprobante + 3)
    

    
    #_Renombra las columnas
    worksheet3.write('B3', 'Tipo ret', header_format_w3)
    worksheet3.write('C3', 'Ret', header_format_w3)
    worksheet3.write('D3', 'BASE RETENCION', header_format_w3)
    worksheet3.write('H3', 'COMPROBANTE', header_format_0)
    worksheet3.write('I3', 'TOTAL FAC', header_format_0)
    worksheet3.write('J3', 'TOTAL PDF', header_format_0)
    worksheet3.write('K3', 'DIFERENCIA', header_format_0)
    worksheet3.write('L3', ' ')
    worksheet3.write(celda_comprobante, 'TOTAL', header_format_tot)
    worksheet3.write('H' + str(filas_df_final + 7), 'COMPROBANTE', header_format_0)
    worksheet3.write('I' + str(filas_df_final + 7), 'NC/ND', header_format_0)
    worksheet3.write('J' + str(filas_df_final + 7), 'TOTAL NC/ND', header_format_0)
    worksheet3.write('K' + str(filas_df_final + 7), 'REFERENCIA', header_format_0)
    worksheet3.write('H' + str(df_comprobantes_ncnd.shape[0] + filas_comprobante + 7), 'TOTAL', header_format_tot)
    
    
    
    #_Ancho de las columnas
    worksheet3.set_column('A:A', 2.3)
    worksheet3.set_column('B:B', 10)
    worksheet3.set_column('C:C', 4)
    worksheet3.set_column('D:D', 20, header_format_4)
    worksheet3.set_column('H:J', 16.3, header_format_4)
    worksheet3.set_column('K:K', 14.29, header_format_4)
    worksheet3.set_column('L:L', 12)
    
    
    
    #Combina celdas y aplica formato (SAP)
    worksheet3.merge_range('B2:D2', 'SAP', header_format_sap)
    
    
    
    #Combina celdas y aplica formato (CONTROL)
    worksheet3.merge_range('H2:K2', 'CONTROL', header_format_control)
    
    
    
    #Combina celdas y aplica formato (CONTROL)
    worksheet3.merge_range(rango_control, 'CONTROL', header_format_control) 
    
    
    
    #Cantidad de filas del cuadro "final_df"
    filas_provincia = len(final_df['Ret']) + 1
    
    
    
    #Se utiliza el formato condicional, ya que nos permite aplicarlo sobre un rango especifico
    worksheet3.conditional_format('B4:D28', {'type' : 'cell',
                                     'criteria' : 'not equal to', 
                                     'value' : '"Error"', 
                                     'format': header_format_pro_borde})
    
    
    
    #Se utiliza el formato condicional, para resaltar los totales de cada columna y sus diferencias
    worksheet3.conditional_format("J" + str(filas_comprobante + 3), {'type' : 'cell',
                                     'criteria' : 'not equal to', 
                                     'value' : '"Error"', 
                                     'format': header_format_total})
    
    
     
    #Se utiliza el formato condicional, para resaltar los totales de cada columna y sus diferencias
    worksheet3.conditional_format("J" + str(filas_comprobante + 11), {'type' : 'cell',
                                     'criteria' : 'not equal to', 
                                     'value' : '"Error"', 
                                     'format': header_format_total})    
    
    
    
    #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
    #Se aplica sobre el cuadro SAP
    worksheet3.conditional_format('A4:A29', {'type' : 'cell',
                                             'criteria' : 'not equal to', 
                                             'value' : '"Error"', 
                                             'format': header_format_3})
    
    
    
    #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
    #Se aplica sobre el cuadro SAP
    worksheet3.conditional_format('E4:E29', {'type' : 'cell',
                                             'criteria' : 'not equal to', 
                                             'value' : '"Error"', 
                                             'format': header_format_2})
    
    
    
    #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
    #Se aplica sobre el cuadro SAP
    worksheet3.conditional_format('B30:D30', {'type' : 'cell',
                                             'criteria' : 'not equal to', 
                                             'value' : '"Error"', 
                                             'format': header_format_5})
    
    
    
    rango_diferencia = "K4:K" + str(filas_comprobante + 3)
    

    #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
    #Se aplica sobre la columna K para resaltar aquellos comprobantes que tienen diferencia entre lo calculado y el .PDF
    worksheet3.conditional_format(rango_diferencia, {'type' : 'cell',
                                             'criteria' : '!=', 
                                             'value' : 0, 
                                             'format': header_format_6})
    
    
    
    rango_detalle = "L4:L" + str(filas_comprobante + 2)
    
    
    #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
    #Se aplica sobre la columna L para resaltar aquellos comprobantes que no tienen el detalle de IIBB
    worksheet3.conditional_format(rango_detalle, {'type' : 'cell',
                                             'criteria' : 'equal to', 
                                             'value' : '"SIN DETALLE"', 
                                             'format': header_format_detalle})
    
    
    
    rango_comp_inex = "K" + str(filas_df_final + 8) + ":K" + str(filas_df_final + 7 + len(df_comprobantes_ncnd))

    
    #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
    #Se aplica sobre la columna K para resaltar aquellos comprobantes que tienen la palabra "COMPROBANTE"
    worksheet3.conditional_format(rango_comp_inex, {'type' : 'cell',
                                             'criteria' : 'equal to', 
                                             'value' : '"COMPROBANTE"', 
                                             'format': header_format_comp_inex})    
    

    
    rango_inex = "L" + str(filas_df_final + 8) + ":L" + str(filas_df_final + 7 + len(df_comprobantes_ncnd))    
    
    
    #Se utiliza el formato condicional, ya es el que nos permite aplicar el formato sobre un rango especifico
    #Se aplica sobre la columna L para resaltar aquellos comprobantes que tienen la palabra "INEXISTENTE"
    worksheet3.conditional_format(rango_inex, {'type' : 'cell',
                                             'criteria' : 'equal to', 
                                             'value' : '"INEXISTENTE"', 
                                             'format': header_format_inex}) 
    
    
    
    #Oculta las lineas de cuadricula de la hoja
    worksheet3.hide_gridlines([1])

    
    #Porcentaje visualizacion de la hoja
    worksheet3.set_zoom(90)    
    
    

######################################
############## Tkinter ###############
######################################        
        
        
root = tk.Tk()

root.withdraw()

#mensaje final
tkinter.messagebox.showinfo("Informacion", "El proceso finalizo correctamente")

root.destroy()